In [6]:
import os
import torch
import torchvision.transforms as transforms
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from utils.dataloaders.dataloader_mnist_single import DataLoaderMNIST
from models.definitions.PocketAutoencoder import PocketAutoencoder
import pandas as pd
from tqdm import tqdm
import itertools

os.chdir('/Users/federicoferoggio/Documents/vs_code/latent-communication')

# Initialize DataFrame with additional columns
loss_dataset = pd.DataFrame(columns=['Dataset', 'Model', 'Seed', 'Epochs', 'Learning Rate', 'Batch Size', 'Loss'])

datasets_list = ['MNIST']
seeds = [1, 2, 3, 4]
paths = ['models/checkpoints/SMALLAE/MNIST/']
dataloader_l = [DataLoaderMNIST]
epochs = [5]
batch_sizes = [64, 128]
learning_rates = [0.01, 0.001]

combinations1 = [datasets_list, seeds, paths, dataloader_l, epochs, batch_sizes, learning_rates]
combinations1 = list(itertools.product(*combinations1))

datasets_list = ['MNIST']
seeds = [3, 4]
paths = ['models/checkpoints/SMALLAE/MNIST/']
dataloader_l = [DataLoaderMNIST]
epochs = [1, 10]
batch_sizes = [32, 64, 128]
learning_rates = [0.01, 0.001, 0.0001]

combinations2 = [datasets_list, seeds, paths, dataloader_l, epochs, batch_sizes, learning_rates]
combinations2 = list(itertools.product(*combinations2))

combinations = combinations1 + combinations2

DEVICE = torch.device("mps")
augmentations = [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
print(len(combinations))

for dataset, seed, paths, dataloader, epochs, batch_size, learning_rate in tqdm(combinations):
    dataloader = dataloader(batch_size=batch_size, transformation=augmentations, seed=seed)
    test_loader = dataloader.get_test_loader()
    train_loader = dataloader.get_train_loader()
    config = {
        'model_name': 'SMLLAE',
        'dataset': dataset,
        'weight_var': 1,
        'weight_mean': 0,
        'seed': seed,
        'batch_size': batch_size,
        'num_epochs': epochs,
        'learning_rate': learning_rate,
        'path': paths
    }
    
    torch.manual_seed(config['seed'])
    model = PocketAutoencoder()
    model.to(DEVICE)
    optimizer = Adam(model.parameters(), lr=config['learning_rate'], weight_decay=1e-4)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
    
    for epoch in range(config['num_epochs']):
        overall_loss = 0
        model.train()  # Set the model to training mode
        
        for batch_idx, (x, _) in enumerate(train_loader):
            x = x.to(DEVICE)
            optimizer.zero_grad()
            loss = model.training_step(x)
            overall_loss += loss.item()
            loss.backward()
            optimizer.step()
        
        avg_loss = overall_loss / (len(train_loader) * batch_size)
        new_row = pd.DataFrame({'Dataset': [config['dataset']],
                                'Model': [config['model_name']],
                                'Seed': [config['seed']],
                                'Epochs': [epoch],
                                'Learning Rate': [config['learning_rate']],
                                'Batch Size': [config['batch_size']],
                                'Loss': [avg_loss]})
        loss_dataset = pd.concat([loss_dataset, new_row], ignore_index=True)
        print("\tEpoch", epoch + 1, "complete!", "\tAverage Loss: ", avg_loss)
    
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient calculation
        test_loss = 0
        for x_test, _ in test_loader:
            x_test = x_test.to(DEVICE)
            test_loss += model.validation_step(x_test).item()
        
        avg_test_loss = test_loss / (len(test_loader) * batch_size)
        scheduler.step(avg_test_loss)  # Update the learning rate based on the test loss
        new_row = pd.DataFrame({'Dataset': [config['dataset']],
                                'Model': [config['model_name']],
                                'Seed': [config['seed']],
                                'Epochs': ['Test'],
                                'Learning Rate': [config['learning_rate']],
                                'Batch Size': [config['batch_size']],
                                'Loss': [avg_test_loss]})
        loss_dataset = pd.concat([loss_dataset, new_row], ignore_index=True)
    
    
    # Save the model
    name = f"{config['dataset']}_{config['model_name']}_{config['learning_rate']}_{config['batch_size']}_{config['num_epochs']}_{config['seed']}.pth"
    print(name)
    path = config['path'] + name
    torch.save(model.state_dict(), path)

loss_dataset.to_csv('models/checkpoints/SMALLAE/losses.csv', index=False)


52


  0%|          | 0/52 [00:00<?, ?it/s]/Users/federicoferoggio/Documents/vs_code/latent-communication/.zeroshot/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
/var/folders/tp/9l7ncvh137x08kdlp_fvr1nw0000gn/T/ipykernel_23416/3579691045.py:87: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  loss_dataset = pd.concat([loss_dataset, new_row], ignore_index=True)


	Epoch 1 complete! 	Average Loss:  0.014295315334219922
	Epoch 2 complete! 	Average Loss:  0.013471051630601763
	Epoch 3 complete! 	Average Loss:  0.013367264171136912
	Epoch 4 complete! 	Average Loss:  0.013359573416546908
	Epoch 5 complete! 	Average Loss:  0.013356479577847254


  2%|▏         | 1/52 [01:30<1:16:56, 90.52s/it]

MNIST_SMLLAE_0.01_64_5_1.pth
	Epoch 1 complete! 	Average Loss:  0.014714868470176515
	Epoch 2 complete! 	Average Loss:  0.013862573777013687
	Epoch 3 complete! 	Average Loss:  0.013591269830436403
	Epoch 4 complete! 	Average Loss:  0.013448886207934382
	Epoch 5 complete! 	Average Loss:  0.01341128998350646


...

MNIST_SMLLAE_0.01_128_10_4.pth
	Epoch 1 complete! 	Average Loss:  0.00748489843979319
	Epoch 2 complete! 	Average Loss:  0.006885871741928653
	Epoch 3 complete! 	Average Loss:  0.0067617408891540096
	Epoch 4 complete! 	Average Loss:  0.006719181147306713
	Epoch 5 complete! 	Average Loss:  0.006695649964905687
	Epoch 6 complete! 	Average Loss:  0.006681848885312772
	Epoch 7 complete! 	Average Loss:  0.006672086936832745
	Epoch 8 complete! 	Average Loss:  0.006663112913065755
	Epoch 9 complete! 	Average Loss:  0.006654698399703767
	Epoch 10 complete! 	Average Loss:  0.006649073192488347


 98%|█████████▊| 51/52 [1:25:17<02:33, 153.25s/it]

MNIST_SMLLAE_0.001_128_10_4.pth
	Epoch 1 complete! 	Average Loss:  0.009374442009894705
	Epoch 2 complete! 	Average Loss:  0.007340980528879649
	Epoch 3 complete! 	Average Loss:  0.0072259333586371915
	Epoch 4 complete! 	Average Loss:  0.007187414782316382
	Epoch 5 complete! 	Average Loss:  0.007134252504856665
	Epoch 6 complete! 	Average Loss:  0.0070492568751499216
	Epoch 7 complete! 	Average Loss:  0.006963831557234975
	Epoch 8 complete! 	Average Loss:  0.00689666945117909
	Epoch 9 complete! 	Average Loss:  0.006850970373836471
	Epoch 10 complete! 	Average Loss:  0.006820131329569354


100%|██████████| 52/52 [1:27:14<00:00, 100.67s/it]

MNIST_SMLLAE_0.0001_128_10_4.pth


In [8]:
import os
import torch
import torchvision.transforms as transforms
from torch.optim import Adam
from utils.dataloaders.dataloader_mnist_single import DataLoaderMNIST
from models.definitions.PocketAutoencoder import PocketAutoencoder
from models.definitions.PocketAutoencoder import PocketAutoencoder

import os

os.chdir("/Users/federicoferoggio/Documents/vs_code/latent-communication")

# Assuming you have already defined and initialized test_loader and DEVICE

losses_per_class = pd.DataFrame(columns=["Model", "Class", "Loss"])

DataLoaders = DataLoaderMNIST
batch_size = 64
augmentations = [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]

dataloader = DataLoaders(batch_size=batch_size, transformation=augmentations, seed=seed)

test_loader = dataloader.get_test_loader()
train_loader = dataloader.get_train_loader()

for file in os.listdir("models/checkpoints/SMALLAE/MNIST/"):
    if file.endswith(".pth"):  # Check if the file is a PyTorch model file
        # Load the model
        model = PocketAutoencoder()
        model.load_state_dict(torch.load("models/checkpoints/SMALLAE/MNIST/" + file))
        print("Model loaded:", file)

        # Iterate through each class (0 to 9)
        for n in range(10):
            desired_class = n  # Specify the class you want to filter
            filtered_samples = []

            # Filter samples from the test loader based on the desired class
            for data, label in test_loader:
                indices = torch.nonzero(label == desired_class, as_tuple=False)
                if indices.numel() > 0:
                    for idx in indices:
                        filtered_samples.append((data[idx], label[idx]))

            test_loss_filtered = 0  # Initialize test loss for the current class

            with torch.no_grad():  # Disable gradient calculation
                for x_test, _ in filtered_samples:
                    test_loss_filtered += model.validation_step(x_test).item()

            # Calculate average reconstruction loss for the current class
            if len(filtered_samples) > 0:
                avg_loss = test_loss_filtered / len(filtered_samples)
            else:
                avg_loss = (
                    0  # Handle the case when there are no samples for the current class
                )

            print("\tTest Loss for class", n, ":", avg_loss)
            # Concatenate the results to the DataFrame
            losses_per_class = pd.concat(
                [
                    losses_per_class,
                    pd.DataFrame({"Model": [file], "Class": [n], "Loss": [avg_loss]}),
                ],
                ignore_index=True,
            )

# Save the results to a CSV file
losses_per_class.to_csv("models/checkpoints/SMALLAE/losses_per_class.csv", index=False)

Model loaded: MNIST_SMLLAE_0.001_128_5_4.pth
	Test Loss for class 0 : 0.8365662784600745


/var/folders/tp/9l7ncvh137x08kdlp_fvr1nw0000gn/T/ipykernel_23416/4225085125.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  losses_per_class = pd.concat(


	Test Loss for class 1 : 1.0393805159871274
	Test Loss for class 2 : 0.8830900924034821
	Test Loss for class 3 : 0.9042004975351957
	Test Loss for class 4 : 0.9444078854655054
	Test Loss for class 5 : 0.9092364298522205
	Test Loss for class 6 : 0.9007483102334566
	Test Loss for class 7 : 0.9739232790957165
	Test Loss for class 8 : 0.9081481354314933
	Test Loss for class 9 : 0.9573835132143306
Model loaded: MNIST_SMLLAE_0.001_64_1_4.pth
	Test Loss for class 0 : 0.8528710160936628
	Test Loss for class 1 : 1.1349865086278201
	Test Loss for class 2 : 0.8843246402088986
	Test Loss for class 3 : 0.9126271761880063
	Test Loss for class 4 : 1.0431147588610405
	Test Loss for class 5 : 0.9216891945477559
	Test Loss for class 6 : 0.8874548055807086
	Test Loss for class 7 : 1.0649184425749203
	Test Loss for class 8 : 0.931528761707537
	Test Loss for class 9 : 1.0597748974978747
Model loaded: MNIST_SMLLAE_0.001_128_10_4.pth
	Test Loss for class 0 : 0.8180888764712275
	Test Loss for class 1 : 0.9218

In [12]:
import os
import torch
import pandas as pd
import torchvision.transforms as transforms
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from models.definitions.PocketAutoencoder import PocketAutoencoder

# Assuming you have already defined and initialized test_loader and DEVICE

os.chdir("/Users/federicoferoggio/Documents/vs_code/latent-communication")
losses_per_class = pd.DataFrame(columns=["Model", "Class", "MSE", "SSIM", "PSNR"])


# Function to calculate SSIM and PSNR
def calculate_ssim_psnr(original, reconstructed, data_range=1.0):
    original_np = original.cpu().numpy().squeeze()
    reconstructed_np = reconstructed.cpu().numpy().squeeze()
    ssim_value = ssim(original_np, reconstructed_np, data_range=data_range)
    psnr_value = psnr(original_np, reconstructed_np, data_range=data_range)
    return ssim_value, psnr_value


# Iterate through each model file in the specified directory
for file in os.listdir("models/checkpoints/SMALLAE/MNIST/"):
    if file.endswith(".pth"):  # Check if the file is a PyTorch model file
        # Load the model
        model = PocketAutoencoder()
        model.load_state_dict(
            torch.load("models/checkpoints/SMALLAE/MNIST/" + file, map_location=DEVICE),
            strict=False,
        )
        model.to(DEVICE)
        print("Model loaded:", file)

        # Iterate through each class (0 to 9)
        for n in range(10):
            desired_class = n  # Specify the class you want to filter
            filtered_samples = []

            # Filter samples from the test loader based on the desired class
            for data, label in test_loader:
                indices = torch.nonzero(label == desired_class, as_tuple=False)
                if indices.numel() > 0:
                    for idx in indices:
                        filtered_samples.append((data[idx], label[idx]))

            mse_loss_filtered = 0  # Initialize MSE loss for the current class
            ssim_loss_filtered = 0  # Initialize SSIM loss for the current class
            psnr_loss_filtered = 0  # Initialize PSNR loss for the current class

            with torch.no_grad():  # Disable gradient calculation
                for x_test, _ in filtered_samples:
                    x_test = x_test.to(DEVICE)
                    x_reconstructed = model(x_test)
                    mse_loss_filtered += torch.nn.functional.mse_loss(
                        x_reconstructed, x_test
                    ).item()
                    ssim_value, psnr_value = calculate_ssim_psnr(
                        x_test, x_reconstructed
                    )
                    ssim_loss_filtered += ssim_value
                    psnr_loss_filtered += psnr_value

            # Calculate average losses for the current class
            num_samples = len(filtered_samples)
            if num_samples > 0:
                avg_mse_loss = mse_loss_filtered / num_samples
                avg_ssim_loss = ssim_loss_filtered / num_samples
                avg_psnr_loss = psnr_loss_filtered / num_samples
            else:
                avg_mse_loss = (
                    0  # Handle the case when there are no samples for the current class
                )
                avg_ssim_loss = 0
                avg_psnr_loss = 0

            print(
                f"\tMetrics for class {n} - MSE: {avg_mse_loss}, SSIM: {avg_ssim_loss}, PSNR: {avg_psnr_loss}"
            )
            # Concatenate the results to the DataFrame
            losses_per_class = pd.concat(
                [
                    losses_per_class,
                    pd.DataFrame(
                        {
                            "Model": [file],
                            "Class": [n],
                            "MSE": [avg_mse_loss],
                            "SSIM": [avg_ssim_loss],
                            "PSNR": [avg_psnr_loss],
                        }
                    ),
                ],
                ignore_index=True,
            )

# Save the results to a CSV file
losses_per_class.to_csv("models/checkpoints/SMALLAE/MNIST/more_metrics.csv", index=False)

Model loaded: MNIST_SMLLAE_0.001_128_5_4.pth
	Metrics for class 0 - MSE: 0.8369951843607183, SSIM: -0.16097904340033933, PSNR: 0.7779342676748042


/var/folders/tp/9l7ncvh137x08kdlp_fvr1nw0000gn/T/ipykernel_23416/2432785473.py:82: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  losses_per_class = pd.concat(


	Metrics for class 1 - MSE: 1.0395673592710284, SSIM: -0.23371325305179277, PSNR: -0.16037558986774278
	Metrics for class 2 - MSE: 0.8835762621935948, SSIM: -0.1835876184230727, PSNR: 0.5506155197559055
	Metrics for class 3 - MSE: 0.90332203318577, SSIM: -0.22636077990095746, PSNR: 0.45570925101892745
	Metrics for class 4 - MSE: 0.9446686668939843, SSIM: -0.2258157755824825, PSNR: 0.2597230016664452
	Metrics for class 5 - MSE: 0.9084933037982393, SSIM: -0.25572115042537275, PSNR: 0.4295579999674039
	Metrics for class 6 - MSE: 0.9010694963324791, SSIM: -0.18473424473568056, PSNR: 0.46948514602135094
	Metrics for class 7 - MSE: 0.9728489723998749, SSIM: -0.26055974667325466, PSNR: 0.1288607795634047
	Metrics for class 8 - MSE: 0.9085603426124526, SSIM: -0.19359579711920039, PSNR: 0.42806539692316664
	Metrics for class 9 - MSE: 0.9579845519202906, SSIM: -0.24348689703740048, PSNR: 0.19584358435079216
Model loaded: MNIST_SMLLAE_0.001_64_1_4.pth
	Metrics for class 0 - MSE: 0.853505191815142